In [9]:
from math import sqrt

def all_dhkl(a, wavelength):
    """ 3x3 representation -> 1x6 (a, b, c, alpha, beta, gamma)"""
    d_min = wavelength/2
    d_hkl = []
    hkl_list = []
    h1 = int(a/d_min)

    for h in range(-h1,h1+1):
        for k in range(-h1,h1+1):
            for l in range(-h1,h1+1):
                sum_d = sqrt(h**2+k**2+l**2)
                if sum_d > 0 and a/sum_d > d_min:
                    d_hkl.append(a/sum_d)
                    hkl_list.append([h,k,l])
    d_hkl = np.array(d_hkl)
    hkl_list = np.array(hkl_list)
    seq = np.argsort(1/d_hkl)
    #print(d_hkl[seq])
    return np.array(hkl_list)[seq,:]

hkl_list = all_dhkl(5.692, 1.54184)
#print(hkl_list)

In [10]:
import numpy as np
from math import asin, sin, cos, exp, pi

def intensity(wavelength, cubic, position, hkl):
    """" Calculate intensity """
    h,k,l = hkl
    d = cubic/sqrt(h**2+k**2+l**2)
    sintheta = wavelength/2/d
    theta = asin(sintheta)
    #print(sintheta, wavelength, sintheta/wavelength, 1/2/d)
    f = atom_scatter(sintheta/wavelength)
    F = structure_factor(f, position, hkl)
    LP = 1/sin(theta)**2/cos(theta)
    P = 1 + cos(2*theta)**2
    I = (np.abs(F))**2*LP*P
    return I

def atom_scatter(d0):
    """ scattering factor """
    f = []
    atom_type = [0,0,0,0,1,1,1,1]
    s = [[4.763, 3.174, 1.267, 1.113, 3.285, 8.842, 0.314, 129.424, 0.676],
         [11.46, 7.196, 6.256, 1.645, 0.01, 1.166, 18.519, 47.778, -9.557]]
    count = 0
    for i in atom_type:
        c = s[i]
        f_tmp = c[0]*np.exp(-c[4]*d0) + c[1]*np.exp(-c[5]*d0) + c[2]*np.exp(-c[6]*d0) + c[3]*np.exp(-c[7]*d0) + c[8]
        f.append(f_tmp)
    #print(f)
    return f
   
def structure_factor(f, pos, hkl):
    """ N*1 array"""
    F = 0
    h,k,l=hkl
    for fj, xyz in zip(f, pos):
        x,y,z = xyz
        #F += fj*(-1)**(2*(h*x + k*y+ l*z))
        F += fj*np.exp(2*pi*(1j)*(h*x + k*y+ l*z))
                    
    return F

In [11]:
pos =  [[0,0,0],
        [0.5,0.5,0],
        [0,0.5,0.5],
        [0.5,0,0.5],
        [0.5,0.5,0.5],
        [0.5,0,0],
        [0,0,0.5],
        [0,0.5,0]]

hkl = [2,2,0]
print(hkl, intensity(1.54184, 5.692, pos, hkl)) # set h+k+l = even number
hkl = [2,1,3]
print(hkl, intensity(1.54184, 5.692, pos, hkl)) # set h+k+l = odd number

[2, 2, 0] 23933.5299323
[2, 1, 3] 1.35062089968e-29


In [12]:
# We can now output the intensities for all hkls
def xrd_by_hkls(wavelength, cubic, pos, hkl_list):
    for hkl in hkl_list:
        h,k,l = hkl
        i = intensity(wavelength, cubic, pos, hkl)
        if i > 1:
            print("[%2d %2d %2d] %6.2f" %(h,k,l,i))
            
xrd_by_hkls(1.54184, 5.692, pos, hkl_list)

[ 1 -1 -1] 3868.66
[ 1  1 -1] 3868.66
[-1 -1 -1] 3868.66
[ 1 -1  1] 3868.66
[ 1  1  1] 3868.66
[-1  1 -1] 3868.66
[-1  1  1] 3868.66
[-1 -1  1] 3868.66
[ 0  2  0] 68297.73
[ 0  0 -2] 68297.73
[-2  0  0] 68297.73
[ 0 -2  0] 68297.73
[ 0  0  2] 68297.73
[ 2  0  0] 68297.73
[-2  2  0] 23933.53
[ 2  2  0] 23933.53
[-2  0  2] 23933.53
[-2 -2  0] 23933.53
[-2  0 -2] 23933.53
[ 0 -2 -2] 23933.53
[ 2 -2  0] 23933.53
[ 2  0 -2] 23933.53
[ 0  2  2] 23933.53
[ 2  0  2] 23933.53
[ 0 -2  2] 23933.53
[ 0  2 -2] 23933.53
[-3  1 -1] 1170.29
[-1 -1  3] 1170.29
[-3  1  1] 1170.29
[ 3 -1  1] 1170.29
[-1 -3 -1] 1170.29
[-3 -1  1] 1170.29
[-1  1  3] 1170.29
[-3 -1 -1] 1170.29
[ 1 -3  1] 1170.29
[ 3  1  1] 1170.29
[ 1  1  3] 1170.29
[ 3 -1 -1] 1170.29
[-1 -3  1] 1170.29
[ 1 -3 -1] 1170.29
[-1  1 -3] 1170.29
[ 3  1 -1] 1170.29
[ 1  1 -3] 1170.29
[ 1 -1 -3] 1170.29
[ 1  3  1] 1170.29
[-1  3 -1] 1170.29
[ 1 -1  3] 1170.29
[-1 -1 -3] 1170.29
[ 1  3 -1] 1170.29
[-1  3  1] 1170.29
[-2 -2  2] 12452.05
[-2  2 -2] 1